<a href="https://colab.research.google.com/github/pedroafleite/data_etl/blob/main/data_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f072740c42fb27c3b3a38e919e276fa76448e0154ecbe0ec01415fe36dcd9649
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
Mounted at /content/drive


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
import json


In [3]:
spark = SparkSession.builder.getOrCreate()

In [15]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/cosmos.jl', lines=True)

In [17]:
json_struct = json.loads(df.to_json(orient="records"))
df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [19]:
df_flat.head()

,id,gtin,created_at,response.gcp.code,response.gcp.description,response.ncm.code,response.ncm.description,response.gtin,response.brand,response.image,response.units,response.status,response.autited,response.description
0,16bde255-988c-466b-9d35-39b706b9fbda,7896040700276,1540236268000,10000424,Detergentes para a roupa,3402.20.00,Preparações acondicionadas para venda a retalho,07896040700276,None,https://cdn-cosmos.bluesoft.com.br/products/78...,"[{'gtin': '7896040700276', 'type': 'Unidade', ...",OK,,LAVA ROUPAS COQUETEL ERVAS
1,96f61c2b-5057-439c-9717-847efe5ea7a5,78909434,1539995834000,None,None,1806.90.00,Outros,00000078909434,FERRERO ROCHER,https://cdn-cosmos.bluesoft.com.br/products/78...,"[{'gtin': '78909434', 'type': 'Unidade', 'pack...",OK,AUDITADO,BOMBOM FERRERO ROCHER 3X16G
2,8ee42cd3-4200-45dc-a10f-6b9e62bc5127,7898917945100,1540442673000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN
3,bda1d82f-cfad-482c-a243-65ae7f04a7bd,27702018072396,1540272188000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN
4,427b4261-d73d-4c13-a085-d9d5bb988c39,7896005279595,1549576419000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN


In [12]:
data

'{"id":"16bde255-988c-466b-9d35-39b706b9fbda","gtin":"07896040700276","response":{"gcp":{"code":"10000424","description":"Detergentes para a roupa"},"ncm":{"code":"3402.20.00","description":"Prepara\\u00e7\\u00f5es acondicionadas para venda a retalho"},"gtin":"07896040700276","brand":null,"image":"https:\\/\\/cdn-cosmos.bluesoft.com.br\\/products\\/7896040700276","units":[{"gtin":"7896040700276","type":"Unidade","packing_size":"1"}],"status":"OK","autited":"","description":"LAVA ROUPAS COQUETEL ERVAS"},"created_at":"2018-10-22 19:24:28"}\n{"id":"96f61c2b-5057-439c-9717-847efe5ea7a5","gtin":"00000078909434","response":{"gcp":{"code":null,"description":null},"ncm":{"code":"1806.90.00","description":"Outros"},"gtin":"00000078909434","brand":"FERRERO ROCHER","image":"https:\\/\\/cdn-cosmos.bluesoft.com.br\\/products\\/78909434","units":[{"gtin":"78909434","type":"Unidade","packing_size":"1"}],"status":"OK","autited":"AUDITADO","description":"BOMBOM FERRERO ROCHER 3X16G"},"created_at":"2018

In [ ]:
list_tags = data['doc']['metadata']['otherTags']

d = []

for i in range(len(list_tags)):
  a = data['doc']['metadata']['otherTags'][i]['tagName']
  b = data['doc']['metadata']['otherTags'][i]['tagValue']
  c = a + ':' + b
  d.append(c)

other_tags = '|'.join(d)
print(other_tags)

otherTag01:A|otherTag02:X


In [ ]:
pandas_df = pd.DataFrame({
    'eventTimestamp': [data['eventTimestamp']],
    'eventSource': [data['eventSource']],
    'eventType': [data['eventType']],
    'eventVersion': [data['eventVersion']],
    'accountId': [data['doc']['metadata']['accountId']],
    'superTag01': [data['doc']['metadata']['superTag01']],
    'superTag02': [data['doc']['metadata']['superTag02']],
    'concatOtherTags': [other_tags],
    'payload': [data['doc']['payload']]
})
df = spark.createDataFrame(pandas_df)
df.show()

+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+
|      eventTimestamp| eventSource|  eventType|eventVersion|           accountId|superTag01|superTag02|     concatOtherTags|    payload|
+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+
|2019-04-02T21:55:57Z|doc.contract|DocSchedule|         1.0|EA2425CF-EF72-40C...|         A|         C|otherTag01:A|othe...|texto teste|
+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+



In [ ]:
df.write.parquet("/content/drive/MyDrive/Colab Notebooks/pyspark/output/proto.parquet")

AnalysisException: ignored

In [ ]:
import datetime

f"{datetime.datetime.now():%Y-%m-%d}"

'2021-10-31'